In [1]:
import tensorflow as tf
from glob import glob
import h5py

from random import seed, shuffle

/home/ubuntu/miniconda3/envs/audiodream/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/miniconda3/envs/audiodream/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/miniconda3/envs/audiodream/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/miniconda3/envs/au

In [2]:
f = h5py.File('./packed_features/unbal_train.h5', 'r')

In [3]:
all_y = f['y']
all_x = f['x']

In [4]:
N = len(all_y)
indices =[i for i in range(N)]

In [5]:
RANDOM_SEED = 999

seed(RANDOM_SEED)
shuffle(indices)

tf.set_random_seed(RANDOM_SEED)

In [6]:
indices = indices[:100000]

print(indices)

[570476, 1511971, 782326, 1326492, 757551, 1218857, 659345, 1284573, 540218, 56684, 255275, 1555421, 1437087, 211727, 817747, 129024, 1499196, 7772, 849434, 983127, 157226, 66237, 983690, 1168796, 712820, 664200, 814569, 592581, 1006915, 652863, 1890770, 868452, 1557529, 1702290, 1816498, 947704, 1979068, 1671741, 1482732, 416801, 1090959, 1431412, 1221030, 947249, 1656316, 262159, 879586, 497517, 942930, 1288075, 1714380, 662818, 845927, 3591, 703736, 1209283, 1180440, 1753906, 397813, 970140, 262837, 82125, 84419, 678084, 176452, 1061749, 1246256, 1550461, 1249447, 1545565, 1256228, 191068, 980877, 1485740, 1832973, 365802, 667063, 559990, 650192, 1460692, 15404, 1656673, 496870, 109203, 1643607, 1523755, 780864, 1119359, 1799465, 1312905, 13265, 944603, 1144096, 1002708, 863839, 272466, 1790980, 1279158, 830809, 1618322, 1451832, 1627591, 225045, 1638235, 1346213, 1441561, 1732771, 386694, 1566167, 1108822, 698840, 1587170, 1887815, 1261931, 2004901, 1541177, 1327593, 1200889, 34048

In [7]:
y = all_y[:,0][indices] # zero because it's the speech index

In [8]:
x = all_x.value[indices] # five because it's the middle slice and I only want one frame

/home/ubuntu/miniconda3/envs/audiodream/lib/python3.7/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


MemoryError: Unable to allocate array with shape (2041789, 10, 128) and data type uint8

In [ ]:
x = (x - 128.0)/255.0

In [ ]:
features_input = tf.keras.layers.Input(shape=x[0].shape)
l = tf.keras.layers.Flatten()(features_input)
l = tf.keras.layers.Dense(64, activation='relu')(l)
l = tf.keras.layers.Dense(32, activation='relu')(l)
l = tf.keras.layers.Dense(1, activation='sigmoid')(l)

model = tf.keras.Model(features_input, l, name='AudioClassifier')

In [ ]:
model.compile('adam', loss='binary_crossentropy')

In [ ]:
model.summary()

In [ ]:
model.fit(x, y, batch_size=32, epochs=50, validation_split=0.1)